In [1]:
import pandas as pd
import numpy as np
import os


In [2]:
labels=pd.read_csv('labels.csv')
labels.head()

,swim_id,stroke
0,eca387ee-efe3-4b52-b4ba-2ad59f9b3c56,Freestyle
1,ecc649a0-e9b7-4d89-bdb4-5446913720bf,Freestyle
2,9c3a0f82-003d-43d7-975a-0e63c2c59657,Freestyle
3,3b3dff00-08d4-4e19-8b56-ac95be0d755e,Freestyle
4,f2c736be-8c1a-4016-9cc9-d7875b40e996,BackStroke


In [6]:
dirs='./combined_2'
files=os.listdir(dirs)
files=[dirs+'/'+f for f in files
       if f.endswith('.csv')]
files[:10]

['./combined_2/0057f3aa-a1ec-4d6a-9776-846f6e8fd0e4.csv',
 './combined_2/00798875-f5bc-4ffe-a4f6-b8c38ffb6a6f.csv',
 './combined_2/00d6b11e-bbcb-4e5b-aadf-5dc7212fd92e.csv',
 './combined_2/01082af2-c81b-407a-a7cd-3cbd58ef860f.csv',
 './combined_2/01189fe4-b286-480e-a724-8f2ca82e2a2f.csv',
 './combined_2/01d1cd74-b3ee-472f-bf85-b84318451509.csv',
 './combined_2/0289f9c2-ef67-45cb-9793-e1c5846970d7.csv',
 './combined_2/02b8a1b3-6a99-4a1e-98d5-dbbc3f1898b6.csv',
 './combined_2/0370272b-09b1-40d4-8d72-6c86765b3d4e.csv',
 './combined_2/03c79948-9b13-47b7-aba4-84619cd8b24b.csv']

In [5]:
df = pd.read_csv( './combined_2/fe4b58da-4dff-4bad-a37b-75f639b13052.csv')
df.columns

Index(['timestamp_1', 'fPressureFront_1', 'fPressureSide_1',
       'acceleration.x_1', 'acceleration.y_1', 'acceleration.z_1',
       'quaternion.i_1', 'quaternion.j_1', 'quaternion.k_1',
       'quaternion.real_1', 'timestamp_2', 'fPressureFront_2',
       'fPressureSide_2', 'acceleration.x_2', 'acceleration.y_2',
       'acceleration.z_2', 'quaternion.i_2', 'quaternion.j_2',
       'quaternion.k_2', 'quaternion.real_2'],
      dtype='object')

In [ ]:

label_lst = []
features=[ 'fPressureFront_1', 'fPressureSide_1',
       'acceleration.x_1', 'acceleration.y_1', 'acceleration.z_1',
       'quaternion.i_1', 'quaternion.j_1', 'quaternion.k_1',
        'fPressureFront_2',
       'fPressureSide_2', 'acceleration.x_2', 'acceleration.y_2',
       'acceleration.z_2', 'quaternion.i_2', 'quaternion.j_2',
       'quaternion.k_2' ]
for file in files:
    df = pd.read_csv(file)
    df_selected = df[features]
    nums = np.array(df_selected)
    print(nums.shape)
    for i in range(200, len(nums)-200, 100):
        data_lst.append(nums[i: i+100,:])
        if "115" in file:
            label_lst.append(0)
        elif "116" in file:
            label_lst.append(1)
        elif "117" in file:
            label_lst.append(2)
        elif "118" in file:
            label_lst.append(3)
        elif "119" in file:
            label_lst.append(3)